## ETL

In [1]:
#Importar librerías necesarias
import os
import ast
import pandas as pd

In [2]:
#Carga del dataset en formato json

rows = []
with open("data\steam_games.json") as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df = pd.DataFrame(rows)

In [3]:
#Exploramos la data
df.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN


In [4]:
df.developer.value_counts()

developer
Ubisoft - San Francisco       1259
SmiteWorks USA, LLC            813
Dovetail Games                 253
KOEI TECMO GAMES CO., LTD.     232
Paradox Development Studio     156
                              ... 
Royal9Games,RoyalJL              1
Salus Games                      1
Class of  2016-2017              1
Pirotexnik                       1
Bidoniera Games                  1
Name: count, Length: 10992, dtype: int64

In [117]:
#Resumen estadístico inicial de los campos numéricos
df.describe()

,discount_price
count,225.000000
mean,11.930533
std,17.492643
min,0.490000
25%,1.390000
50%,4.190000
75%,22.660000
max,139.990000


In [118]:
#Observamos número de registros, valores nulos y tipos de columnas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   discount_price  225 non-null    float64
 8   reviews_url     32133 non-null  object 
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  bool   
 12  id              32133 non-null  object 
 13  developer       28836 non-null  object 
 14  sentiment       24953 non-null  object 
 15  metascore       2677 non-null   object 
dtypes: bool(1), float64(1), object(14)
memory usage: 3.7+ MB


#### Transformación de la columna "price".

In [119]:
#Remplazamos por cero las cadenas "Free To Play", "Free to Play", "Free".
reemplazos = {"Free To Play": "0", "Free to Play": "0", "Free": "0"}
df["price"] = df["price"].replace(reemplazos)

In [120]:
#Convertimos la columna "price" de "object" en "float" para poder realizar operaciones matemáticas
df['price'] = pd.to_numeric(df['price'], errors= 'coerce', downcast = 'float')

In [121]:
#Cuenta los valores no numéricos de la columna "price"
df['price'].isna().sum()

1400

In [122]:
#Elima los valores no numéricos de la columna "price"
df.dropna(subset=['price'], inplace=True)

#### Transformación de la columna "release_date".

In [123]:
#Transforma la columna release_date de tipo "object" a "datetime" para poder hacer análisis y extraer sus componentes
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

#Extracción del año de la columna release_date para operar con más eficiencia
df['year'] = df['release_date'].dt.year

In [124]:
#Contar nulos
df['year'].isnull().sum()

1976

In [125]:
#Eliminar las filas en las que la columna "year" tenga valores nulos. 
#Nos piden crear funciones en las que solo extraemos datos a partir de los años y no pueden ser nulos. 
df.dropna(subset=['year'], inplace=True)

#Cambiamos el tipo de dato de la columna a entero.
df['year'] = df['year'].astype(int)

#Eliminamos la columna porque ya extrajimos el año, que era el valor necesario para hacer las operaciones requeridas. 
df.drop(columns= ["release_date"], inplace=True)

#### Transformación de la columna "metascore".

In [126]:
# Cambiamos la columna de tipo de dato "object" a "float".
df['metascore'] = pd.to_numeric(df['metascore'], errors='coerce')

In [127]:
# Imputamos los valores Nan de la columna con cero.
# Debemos imputar datos porque es una columna que utilizaremos para operaciones y eliminar los NaN implicaría
# borrar el 91% de los valores lo cual afectaría el conjunto de datos. 
df['metascore'] = df['metascore'].fillna(0)

In [128]:
#Calcula el porcentaje de valores iguales a 0 tiene la columna "metascore". 
cantidad_ceros = df['metascore'].eq(0).sum()
total_filas = df['metascore'].count()
porcentaje_ceros = (cantidad_ceros / total_filas) * 100
porcentaje_ceros

91.23057129941931

#### Transformación de la columna "sentiment".

In [129]:
# Eliminamos los nulos porque nos exigen utilizar registros categorizados. 
df.dropna(subset=['sentiment'], inplace=True)

In [130]:
#Contamos las categorías de la columna.
df["sentiment"].value_counts() 

sentiment
Mixed                      3747
Very Positive              3543
Positive                   2849
Mostly Positive            2489
1 user reviews             2275
2 user reviews             1566
3 user reviews             1081
4 user reviews              849
5 user reviews              743
Mostly Negative             738
6 user reviews              643
7 user reviews              518
8 user reviews              458
9 user reviews              419
Overwhelmingly Positive     290
Negative                    113
Very Negative                26
Overwhelmingly Negative       7
Name: count, dtype: int64

In [131]:
ser_reviews_count = df[df['sentiment'].str.contains(r'\d+ user reviews', case=False)].shape[0]
ser_reviews_count

8552

#### Columna "id".

In [132]:
df["id"].isnull().sum()

1

In [133]:
#Eliminamos el valor nulo encontrado. Es una anomalía dentro de la columna. 
df = df.dropna(subset=['id'])

#### Columna "discount_price".

In [134]:
total_rows = len(df)

# Paso 2: Calcular la cantidad de filas con valores nulos en la columna "discount_price"
null_count = df["discount_price"].isnull().sum()

# Paso 3: Calcular el porcentaje de nulos en la columna "discount_price"
percentaje_nulos = (null_count / total_rows) * 100

print(f"Porcentaje de valores nulos en 'discount_price': {percentaje_nulos:.2f}%")

Porcentaje de valores nulos en 'discount_price': 99.56%


* * La columna discount_price	tiene un 99.54% de valores nulos. No se usará para las funciones que crearemos para la APi y la cantidad de datos válidos es muy baja para poder considerarla adecuada para el modelo de machine learning. Por estas razones la eliminamos. 

In [135]:
# Eliminar columna
df.drop(columns= ["discount_price"], inplace=True)

Eliminación de nulos en columnas "genres" y "specs".

In [136]:
#Conteo de nulos
df["genres"].isnull().sum()

596

In [137]:
#Eliminamos nulos porque para las funciones nos piden registros con valores. 
#Además, es una columna importante para el modelamiento de ML y no puede tener nulos.  
df = df.dropna(subset=["genres"])

In [138]:
df["specs"].isnull().sum()

229

In [139]:
df = df.dropna(subset=["specs"])

#### Eliminación de columnas 

Criterios de eliminación:
* Nos piden seis funciones para las que no necesitaremos las columnas "publisher", "tags", "developer","reviews_url", "url". Por otra parte, solo la columna "developer" podría tener influencia en nuestra variable target, pero hay otras variables independientes más importantes cómo "genres", "year, "early_access".
* La columna "title" cumple la misma función que "app_name", por lo que eliminamos la primera. 

In [140]:
df.drop(columns= ["publisher", "tags", "developer", "reviews_url", "url", "title"], inplace=True)

In [141]:
# csv_path = os.path.join("data", "steam.csv")
# df.to_csv(csv_path, index=False)

In [142]:
# Exportar el DataFrame como un archivo CSV   ### ojo
df.to_csv('steam.csv', index=False)

#### Columnas anidadas "genres" y "specs".

* Los valores de "genres" se encuentran en forma de lista. Crearemos un dataset aparte que contenga los géneros únicos y le asignaremos un id. También insertaremos en el dataset actual una columna con los id de los géneros. 

In [143]:
#Cargamos el nuevo dataset y continuamos realizando cambios
steam = pd.read_csv("steam.csv")

In [144]:

# Convertir las cadenas de texto en listas de Python
steam['genres'] = steam['genres'].apply(ast.literal_eval)

# Obtener todos los géneros de todas las listas en la columna 'genres'
all_genres = [genre for genres_list in steam['genres'] for genre in genres_list]

# Obtener géneros únicos eliminando duplicados
unique_genres = list(set(all_genres))

print(unique_genres)

['Racing', 'Audio Production', 'Indie', 'Photo Editing', 'Animation &amp; Modeling', 'Simulation', 'Education', 'Early Access', 'Video Production', 'Strategy', 'Sports', 'Adventure', 'Web Publishing', 'Design &amp; Illustration', 'Action', 'Utilities', 'Casual', 'Software Training', 'Massively Multiplayer', 'Free to Play', 'RPG']


In [145]:
#Tenemos un primer DataFrame con los géneros únicos
df_unique_genres = pd.DataFrame({'unique_genres': unique_genres})
df_unique_genres

,unique_genres
0,Racing
1,Audio Production
2,Indie
3,Photo Editing
4,Animation &amp; Modeling
5,Simulation
6,Education
7,Early Access
8,Video Production
9,Strategy


In [146]:
# Creamos un diccionario
genre_to_id = {genre: idx for idx, genre in enumerate(unique_genres, start=1)}

genre_to_id

{'Racing': 1,
 'Audio Production': 2,
 'Indie': 3,
 'Photo Editing': 4,
 'Animation &amp; Modeling': 5,
 'Simulation': 6,
 'Education': 7,
 'Early Access': 8,
 'Video Production': 9,
 'Strategy': 10,
 'Sports': 11,
 'Adventure': 12,
 'Web Publishing': 13,
 'Design &amp; Illustration': 14,
 'Action': 15,
 'Utilities': 16,
 'Casual': 17,
 'Software Training': 18,
 'Massively Multiplayer': 19,
 'Free to Play': 20,
 'RPG': 21}

In [147]:
#Convertimos el diccionario en un DataFrame 
df_genre_to_id = pd.DataFrame.from_dict(genre_to_id, orient='index', columns=['id'])

# csv_path2 = os.path.join("data", "genre_to_id.csv")
# df.to_csv(csv_path2, index=False)

# Guardar el DataFrame en un archivo CSV
df_genre_to_id.to_csv('genre_to_id.csv', index_label='genre') 

In [148]:
# Función para mapear los géneros a sus respectivos IDs
def map_genres_to_ids(genres_list):
    return [genre_to_id[genre] for genre in genres_list]

# Aplicar la función a la columna 'genres' para obtener la columna 'genre_id'
steam['genre_id'] = steam['genres'].apply(map_genres_to_ids)


* Aplicamos el mismo procedimiento a la columna "specs".

In [149]:
# Convertir las cadenas de texto en listas de Python
steam['specs'] = steam['specs'].apply(ast.literal_eval)

# Obtener todos los géneros de todas las listas en la columna 'genres'
all_specs = [spec for specs_list in steam['specs'] for spec in specs_list]

# Obtener géneros únicos eliminando duplicados
unique_specs = list(set(all_specs))

print(unique_specs)

['MMO', 'Cross-Platform Multiplayer', 'Includes Source SDK', 'Steam Trading Cards', 'Steam Leaderboards', 'Valve Anti-Cheat enabled', 'Local Co-op', 'Mods', 'In-App Purchases', 'Online Co-op', 'Game demo', 'Mods (require HL2)', 'Downloadable Content', 'Stats', 'Shared/Split Screen', 'Full controller support', 'Single-player', 'Steam Turn Notifications', 'Steam Cloud', 'Multi-player', 'Partial Controller Support', 'Commentary available', 'SteamVR Collectibles', 'Online Multi-Player', 'Includes level editor', 'Local Multi-Player', 'Steam Achievements', 'Captions available', 'Steam Workshop', 'Co-op']


In [150]:
#Tenemos un primer DataFrame con los géneros únicos
df_unique_specs = pd.DataFrame({'unique_specs': unique_specs})
df_unique_specs

,unique_specs
0,MMO
1,Cross-Platform Multiplayer
2,Includes Source SDK
3,Steam Trading Cards
4,Steam Leaderboards
5,Valve Anti-Cheat enabled
6,Local Co-op
7,Mods
8,In-App Purchases
9,Online Co-op


In [151]:
# Creamos un diccionario
spec_to_id = {spec: idx for idx, spec in enumerate(unique_specs, start=1)}

spec_to_id

{'MMO': 1,
 'Cross-Platform Multiplayer': 2,
 'Includes Source SDK': 3,
 'Steam Trading Cards': 4,
 'Steam Leaderboards': 5,
 'Valve Anti-Cheat enabled': 6,
 'Local Co-op': 7,
 'Mods': 8,
 'In-App Purchases': 9,
 'Online Co-op': 10,
 'Game demo': 11,
 'Mods (require HL2)': 12,
 'Downloadable Content': 13,
 'Stats': 14,
 'Shared/Split Screen': 15,
 'Full controller support': 16,
 'Single-player': 17,
 'Steam Turn Notifications': 18,
 'Steam Cloud': 19,
 'Multi-player': 20,
 'Partial Controller Support': 21,
 'Commentary available': 22,
 'SteamVR Collectibles': 23,
 'Online Multi-Player': 24,
 'Includes level editor': 25,
 'Local Multi-Player': 26,
 'Steam Achievements': 27,
 'Captions available': 28,
 'Steam Workshop': 29,
 'Co-op': 30}

In [152]:
#Convertimos el diccionario en un DataFrame 
df_spec_to_id = pd.DataFrame.from_dict(spec_to_id, orient='index', columns=['id'])

# csv_path3 = os.path.join("data", "spec_to_id.csv")
# df.to_csv(csv_path3, index=False)

# Guardar el DataFrame en un archivo CSV
df_spec_to_id.to_csv('spec_to_id.csv', index_label='spec') 

In [153]:
# Función para mapear los géneros a sus respectivos IDs
def map_specs_to_ids(specs_list):
    return [spec_to_id[spec] for spec in specs_list]

# Aplicar la función a la columna 'genres' para obtener la columna 'genre_id'
steam['spec_id'] = steam['specs'].apply(map_specs_to_ids)

In [154]:
# csv_path = os.path.join("data", "steam.csv")
# df.to_csv(csv_path, index=False)


#Guardamos los últimos cambios en el csv limpio                         ###ojo
steam.to_csv('steam.csv', index=False)

In [72]:
steam.head()

,genres,app_name,specs,price,early_access,id,sentiment,metascore,year,genre_id,spec_id
0,"[Free to Play, Indie, RPG, Strategy]",Ironbound,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,Mostly Positive,0.0,2018,"[20, 3, 21, 10]","[17, 20, 24, 2, 27, 4, 9]"
1,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290,Mostly Positive,0.0,2017,"[17, 20, 3, 6, 11]","[17, 20, 24, 9, 14]"
2,"[Action, Adventure, Simulation]",Battle Royale Trainer,"[Single-player, Steam Achievements]",3.99,False,772540,Mixed,0.0,2018,"[15, 12, 6]","[17, 27]"
3,"[Adventure, Casual, Indie, Simulation, Strategy]",The first thrust of God,"[Single-player, Steam Achievements, Steam Cloud]",1.59,False,773650,1 user reviews,0.0,2018,"[12, 17, 3, 6, 10]","[17, 27, 19]"
4,"[Action, Adventure, Indie]",The Warrior Of Treasures,[Single-player],0.99,False,768060,3 user reviews,0.0,2018,"[15, 12, 3]",[17]


In [43]:
steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21528 entries, 0 to 21527
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        21528 non-null  object 
 1   app_name      21528 non-null  object 
 2   specs         21528 non-null  object 
 3   price         21528 non-null  float64
 4   early_access  21528 non-null  bool   
 5   id            21528 non-null  int64  
 6   sentiment     21528 non-null  object 
 7   metascore     21528 non-null  float64
 8   year          21528 non-null  int64  
 9   genre_id      21528 non-null  object 
 10  spec_id       21528 non-null  object 
dtypes: bool(1), float64(2), int64(2), object(6)
memory usage: 1.7+ MB
